In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

binary_image = cv2.imread("road_dil_grey.png", 0)


# Perform skeletonization using the morphological operation "cv2.ximgproc.thinning"
skeleton = np.zeros(binary_image.shape, np.uint8)
skeleton = cv2.ximgproc.thinning(
    binary_image, skeleton, cv2.ximgproc.THINNING_GUOHALL)

# Detect intersections by finding the pixels that have a total of more than two non-zero neighbors
kernel = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], np.uint8)
intersections = cv2.filter2D(skeleton, -1, kernel)
intersections[intersections > 2] = 255
intersections[intersections <= 2] = 0

# Mark the intersections in white
output_image = np.zeros(
    (binary_image.shape[0], binary_image.shape[1], 3), np.uint8)
output_image[:, :, 0] = intersections

# threshold the skeleton to get the edges
blue_threshold = 25
output_image[intersections > blue_threshold] = 255

#smoothen the output_image
# output_image = cv2.GaussianBlur(output_image, (5, 5), 0)

cv2.imwrite("skeletonized_roadmap.png", output_image)


In [ ]:
import numpy as np
import networkx as nx
import cv2


def find_nodes(skeleton):
    nodes = np.zeros_like(skeleton)
    for i in range(1, skeleton.shape[0] - 1):
        for j in range(1, skeleton.shape[1] - 1):
            neighbors = skeleton[i-1:i+2, j-1:j+2]
            if (skeleton[i, j] == 1 and
                    np.sum(neighbors) - 1 in [1, 3, 4]):
                nodes[i, j] = 1
    return nodes


def find_edges(skeleton, nodes):
    edges = []
    for i in range(nodes.shape[0]):
        for j in range(nodes.shape[1]):
            if nodes[i, j]:
                node_id = i * nodes.shape[1] + j
                for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    x, y = i + dx, j + dy
                    if skeleton[x, y]:
                        edge = (node_id, x * nodes.shape[1] + y)
                        edges.append(edge)
    return edges


def skeletonize(image):
    size = np.size(image)
    skel = np.zeros(image.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))

    done = False
    while not done:
        eroded = cv2.erode(image, element)
        temp = cv2.dilate(eroded, element)
        temp = cv2.subtract(image, temp)
        skel = cv2.bitwise_or(skel, temp)
        image = eroded.copy()

        zeros = size - cv2.countNonZero(image)
        if zeros == size:
            done = True
    return skel


def nefi_graph(image):
    skeleton = skeletonize(image).astype(np.uint8)
    nodes = find_nodes(skeleton)
    edges = find_edges(skeleton, nodes)
    graph = nx.Graph()
    graph.add_nodes_from(np.argwhere(nodes == 1))
    graph.add_edges_from(edges)
    return graph, nodes


# Example usage:
img = cv2.imread("skeletonized_roadmap.png", cv2.IMREAD_GRAYSCALE)
graph, nodes = nefi_graph(img)

# Plot the graph
node_coords = np.argwhere(nodes == 1)
edges = nx.to_edgelist(graph)
output_img = np.zeros_like(img)
for x1, y1, x2, y2 in edges:
    cv2.line(output_img, (y1, x1), (y2, x2), (255, 255, 255), 1)
for x, y in node_coords:
    cv2.circle(output_img, (y, x), 2, (0, 0, 255), -1)
cv2.imwrite("output_graph.png", output_img)
